# Zadanie 1

#### 1. Zapisz łączne prawdopodobieństwo jako tablicę numpy o odpowiednich wymiarach.

In [2]:
import numpy as np
P = np.array([[[0.108, 0.012], [0.072, 0.008]], [[0.016, 0.064], [0.144, 0.576]]]) #indeksy - [cav][toothache][catch]
print(f'P: {P}')

P: [[[0.108 0.012]
  [0.072 0.008]]

 [[0.016 0.064]
  [0.144 0.576]]]


#### 2. Oblicz P(Toothache) (przydatne funkcje: np.sum).

In [3]:
P_cav_too = np.sum(P, axis = -1)#axis 0 - kolumnami , 1 - wierszami -> najpierw pozbywamy się osi 0, a potem osi 2
P_too = np.sum(P_cav_too, axis = 0) #Marginalizacja to jest sumowanie po tych zmiennych, których chcemy się pozbyć
print(f'P_tot: {P_too}')

P_tot: [0.2 0.8]


#### 3. Oblicz P(Cavity).

In [4]:
P_cav = np.sum(P_cav_too, axis=-1)
print(f'P_cav: {P_cav}')

P_cav: [0.2 0.8]


#### 4. Oblicz P(Toothache|Cavity). Wynik zapisz tak, aby indeks zmiennej Toothache był pierwszym wymiarem (przydatne funkcje: np.transpose).

In [5]:
P_too_giv_cav = np.transpose(P_cav_too) / P_cav
print(f'P_too_giv_cav: {P_too_giv_cav}')

P_too_giv_cav: [[0.6 0.1]
 [0.4 0.9]]


#### 5. Oblicz P(Cavity|toothache∨catch), gdzie toothache∨catch oznacza występowanie bólu zęba lub zakleszczenia.

In [6]:
P_cav_giv_too_or_catch = (P_cav - P[:,1,1]) / (np.sum(P, axis=(0,1,2)) - np.sum(P[:,1,1], axis=0))
print(f'P_cav_giv_too_or_catch: {P_cav_giv_too_or_catch}')

P_cav_giv_too_or_catch: [0.46153846 0.53846154]


#### 6. Jak zależy wielkość tablicy z pełnym rozkładem prawdopodobieństwa od liczby zmiennych, zakładając, że zmienne te są binarne (każda może przyjąć 2 wartości)?

##### Odp: Wielkość tablicy -> 2^liczba zmiennych^. W tym przypadku 2^3^ = 8

#### 7. Ile pamięci operacyjnej byłoby potrzebne do przechowania takiej tablicy dla 32 zmiennych, zapisując liczby jako 32 bitowy float?

##### Odp: ilosc_pamieci = 32 * 2^32^ bitów = 2^37^ bajtów = 137438953472 bitów = 16384 MB = 16GB

#### 8. Jak obliczyć P(Cavity|Toothache,Catch) nie znając pełnego rozkładu, a dysponując jedynie P(Toothache,Catch|Cavity) oraz P(Cavity)? Zaimplementuj i przetestuj rozwiązanie (zasymuluj dostępność P(Toothache,Catch|Cavity) oraz P(Cavity) obliczając te rozkłady z rozkładu łącznego). Jakie jest prawdopodobieństwo, że pacjent ma próchnicę, jeśli boli go ząb i wiertło nie zakleszczyło się w zębie? A jakie jeśli boli go ząb i wiertło zakleszczyło się w zębie?

In [7]:
# (1) Regula iloczynu: P(a,b) = P(a|b)P(b)
# (2) Regula Bayesa: P(b|a) = P(a|b)*P(b)/P(a)
# Zatem z (2) P(Cav|Too,Cat)=P(Too,Cat|Cav)*P(Cav)/P(Too,Cat)
# Z (1) P(Cav|Too,Cat)=P(Too,Cat,Cav)/P(Too,Cat)
# Stąd P(Too,Cat,Cav)=P(Too,Cat|Cav)*P(Cav)

P_too_cat_giv_cav = P/np.reshape(P_cav, (2, 1, 1))
# P_too_cat_giv_cav = P/np.transpose(P_cav, axes=(-1, -2))
print("P_too_cat_giv_cav ", P_too_cat_giv_cav)

P_too_cat = np.sum(P, axis=0)
print("P_too_cat: ", P_too_cat)

P_cav_giv_too_cat = P/P_too_cat
print("P_cav_giv_too_cat ",P_cav_giv_too_cat)
# Wspolczynnik normalizujacy alfa, suma wartosci w rozkladzie musi sie rownac 1, czyli podzielic przez sume wszystkich elementow (1/suma elementow w rozkladzie)

P_too_cat_giv_cav  [[[0.54 0.06]
  [0.36 0.04]]

 [[0.02 0.08]
  [0.18 0.72]]]
P_too_cat:  [[0.124 0.076]
 [0.216 0.584]]
P_cav_giv_too_cat  [[[0.87096774 0.15789474]
  [0.33333333 0.01369863]]

 [[0.12903226 0.84210526]
  [0.66666667 0.98630137]]]


#### 9. Czy zmienne Toothache i Catch są od siebie niezależne? Co z niezależnością warunkową, mając dane Cavity?

##### Odp: Są zależne, chyba, że mamy do czynienia  próchnicą, wówczas mamy zmienne są niezależne.

#### 10. Wykorzystaj te zależności, aby obliczyć P(Cavity|Toothache,Catch) mając dane P(Toothache|Cavity), P(Catch|Cavity) oraz P(Cavity).

In [8]:
# Korzystając z zadania 8. wiemy, że:
# 1) P(Cav | Too, Cat) = P(Too, Cat | Cav) * P(Cav) / P(Too,Cat)
# 2) P(Cav | Too, Cat) = P(Cav, Too, Cat) / P(Too, Cat)
# Zmienne Toothache i Catch są niezależne pod warunkiem Cavity, zatem P(Too, Cat | Cav) = P(Too | Cav) * P(Catch | Cavity)
# P(Cav | Too, Cat) = P(Too | Cav) * P(Cat | Cav) * P(Cav) / P(Too, Cat)
# P(Too | Cav) = P_toothache_giv_cavity, P(Cat | Cav) = P_cat_git_cavity, P(Cav) = P_cav, P(Too, Cat) = P_too_cat

# P(Cav|Too,Cat) = P(Too|Cav)*P(Cat|Cav)*P(Cav)/P(Too,Cat) = P(Too,Cat|Cav)*P(Cav)/P(Too,Cat)
P_Cav_giv_too_cat_10 = P_too_cat_giv_cav * np.reshape(P_cav, (2, 1, 1)) / P_too_cat
print("P(Cav | Too, Cat) zadanie 10.: ", P_Cav_giv_too_cat_10)

P(Cav | Too, Cat) zadanie 10.:  [[[0.87096774 0.15789474]
  [0.33333333 0.01369863]]

 [[0.12903226 0.84210526]
  [0.66666667 0.98630137]]]


#### 11. Jak rozłożyć pełen rozkład prawdopodobieństwa za pomocą danych z poprzedniego podpunktu?

##### Odp: P(Cav, Too, Cat) = P(Cav|Too,Cat)*P(Too,Cat) = P(Too|Cav)*P(Cat|Cav)*P(Cav)

#### 11. Ile pamięci potrzeba do przechowywania pełnego rozkładu, rozłożonego na czynniki, jeśli mamy 31 niezależnych warunkowo zmiennych i jedną zmienną separującą te zmienne?

##### Odp: Mamy 31 niezależnych zmiennych. Zakładamy, że zmienne mają wartości binarne (0/1), zatem będzie 31 tablic dwuelementowych z prawdopodobieństwem warunkowym (pod warunkiem zmiennej separującej).
Jeśli zmienna sepacująca = B, to mamy P(A1|B), P(A2|B)...P(A31|B)
Mamy też tablicę dwuelementową z prawdopodobieństwem zmiennej separującej P(B)
Iloczyn prawdopodobieństw = pełen rozkład

Ile zmiennych: 31*2+2 (31 tablic dwuelementowych + tablica dwuelementowa)
Pamięć = rozmiar zmiennych (np. 32 bity) * Ile zmiennych